In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def shadow_remove(img):
    rgb_planes = cv2.split(img)
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_norm_planes.append(norm_img)
    shadowremov = cv2.merge(result_norm_planes)
    return shadowremov

In [3]:
# def hog_preprocessing(image):
#     image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     resized_image = cv2.resize(image,(500,500))
#     shad_image = shadow_remove(resized_image)
#     grayscale_image = cv2.cvtColor(shad_image, cv2.COLOR_BGR2GRAY)
#     filtered_image  = cv2.GaussianBlur(grayscale_image, (3, 3), 1.0)
#     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
#     normalized_image = clahe.apply(filtered_image)
#     img_array = np.array(normalized_image, dtype=np.float32) / 255.0
#     corrected_array = np.power(img_array, 0.5)
#     corrected_image = np.uint8(corrected_array * 255.0)
#     resized_image = cv2.resize(corrected_image,(128,128))
#     return resized_image

In [4]:
def hog_preprocessing(image):
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    filtered_image  = cv2.GaussianBlur(image, (3, 3), 1.0)
    resized_image = cv2.resize(filtered_image,(500,500))
    hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2HSV)
    #=====================================Kmean
    Z = np.float32(hsv_image.reshape((-1,3)))
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 4
    _,labels,centers = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    labels = labels.reshape((hsv_image.shape[:-1]))
    #==========================range
    light_skin = (0, 50, 50 )
    dark_skin = (25, 200, 255)
    # result=resized_image
    # for i, c in enumerate(centers):
    #     if all(int(c[j]) in range(light_skin[j],dark_skin[j]) for j in range(3)):
    #             mask = cv2.inRange(labels, i, i)
    #             mask = np.dstack([mask]*3) # Make it 3 channel
    #             result = cv2.bitwise_and(resized_image, mask)
    mask = cv2.inRange(hsv_image, light_skin, dark_skin)
    result = cv2.bitwise_and(resized_image, resized_image, mask=mask)
    resized_image = cv2.resize(result,(128,128))
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2GRAY)
    return grayscale_image